In [128]:
import random
import numpy as np
import matplotlib.pyplot as plt
import math
from copy import deepcopy
from GA_functions import *

#### Encode the problem
* Each item is encoded with its: ( _benefit_, _volume_ )
* `CAPACITY` of the knapsack can be adjusted.
* `NUM_GENES` is the number of genes each individual has.

In [2]:
items = np.array([(5,3), (6,2), (1,4), (9,5), (2,8), (8,9), (4,10), (3,1), (7,6), (10,7)])

CAPACITY = 20
NUM_GENES = len(items)

#### Task 1: Implement a steady state GA with tournament selection

* You will need a population of N individuals. 
* This should be encoded as a matrix of N genotypes, wherein each genotype encodes one possible solution to knapsack problem. 

In [32]:
N = 100
population_1 = random_population(population_size = N, length=NUM_GENES, vectorized=True)

* A genotype → phenotype mapping.

In [253]:
def print_population(population):
    i = 0
    for individual in population:
        print(i, ": ", individual, " score: ", fitness(individual))
        i += 1

* A fitness function . We need a way to evaluate how good each phenotype is as a potentialsolution to the card-sorting problem. 

In [254]:
help(fitness)
fitness(population_1[0])

Help on function fitness in module GA_functions:

fitness(individual)
    # #### Fitness
    # * Given an individual, evaluate its fitness.



-27

* A mutation operator.

In [255]:
individual_1 = population_1[0]
individual_1_mutated = mutate(individual_1)

print(individual_1)
print(individual_1_mutated)

[1 1 1 1 0 0 1 1 1 1]
[1 1 1 1 0 0 1 0 1 1]


* Tournament Selection

In [256]:
def battle(individual_1, individual_2):
    '''
    Returns the individual with the higher fitness score.
    '''
    if fitness(individual_1) > fitness(individual_2):
        return individual_1
    else:
        return individual_2
    
# Test battle
population_1 = random_population(population_size = N, length=NUM_GENES, vectorized=True)
individual_1 = population_1[1]
individual_2 = population_1[2]
print(individual_1, " score: ", fitness(individual_1))
print(individual_2, " score: ", fitness(individual_2))
winner = battle(individual_1, individual_2)
print(winner, " WINNER SCORE: ", fitness(winner))

[1 0 1 1 0 1 1 1 0 1]  score:  -36
[0 1 0 0 1 1 1 1 0 0]  score:  -17
[0 1 0 0 1 1 1 1 0 0]  WINNER SCORE:  -17


In [310]:
def battle(population):
    scores = matrix_to_scores(population)
    winner_indices = np.ones(int(len(population)/2), dtype=int)
    
    # use j to iterate over winner_indices
    j = 0
    
    # use i to iterate over tuples of indivuduals.
    
    # ex:
    # when i = 0: the battle is between population[0] and population[1]
    # the winner's index is recorded at winner_indices[0]
    
    # when i = 2 the battle is between population[2] and population[3]
    # the winner's index is recored at winner_indices[1]
    # ...

    for i in range(0, len(population), 2):
        if scores[i] > scores[i+1]:
            winner_indices[j] = i
        else:
            winner_indices[j] = i+1
            
        j += 1
        
    return population[winner_indices, :]
    
N = 8
population_1 = random_population(population_size = N, length=NUM_GENES, vectorized=True)

print_population(population_1)
battle(population_1)

0 :  [1 0 1 0 1 1 0 1 1 0]  score:  -18
1 :  [1 1 0 0 1 0 0 0 1 0]  score:  19
2 :  [0 0 0 0 0 0 0 1 0 0]  score:  -16
3 :  [0 0 0 0 1 1 1 0 0 0]  score:  -14
4 :  [0 1 0 1 1 1 1 0 0 1]  score:  -45
5 :  [1 1 1 0 0 1 0 1 0 0]  score:  22
6 :  [1 1 1 0 1 1 1 1 1 0]  score:  -56
7 :  [0 1 0 1 0 1 1 0 0 0]  score:  3


array([[1, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 1, 1, 0, 0, 0]])

In [318]:
powers_of_two = [2**i for i in range(1000)]

def tournament(population):
    if len(population) not in powers_of_two:
        return "Error: In order to have a propert torunament, the size of the population must be a power of two."
    else:
        while len(population) != 1:
            #print("New Battle!")
            population = battle(population)
            #print_population(population)
            #print("Individuals Remaining: ", len(population))
    
    # return the champion
    return population[0]

N = 32
population_1 = random_population(population_size = N, length=NUM_GENES, vectorized=True)
tournament(population_1)

array([1, 1, 1, 1, 0, 0, 0, 0, 0, 1])

In [333]:
def multiple_tournaments(num_tournaments=10):
    winners = []
    for i in range(num_tournaments):
        population_i = random_population(population_size = num_tournaments, length=NUM_GENES, vectorized=True)
        winner_i = tournament(population_i)
        winners.append([winner_i, fitness(winner_i)])
    return winners

mul_winners = multiple_tournaments(16)
for winner in mul_winners:
    print(winner[0], " score: ", winner[1])
    
    
tournament(mul_winners)

[1 0 1 1 0 1 0 1 0 0]  score:  18
[1 1 0 1 0 1 0 1 0 0]  score:  31
[1 0 0 1 0 0 0 1 1 1]  score:  26
[1 1 0 0 0 1 0 1 0 1]  score:  24
[1 1 0 1 0 1 0 0 0 0]  score:  27
[1 1 0 0 1 0 0 0 1 0]  score:  19
[1 1 0 0 0 0 0 1 1 1]  score:  30
[1 1 1 1 0 0 0 0 1 0]  score:  28
[1 0 1 1 0 1 0 0 0 0]  score:  19
[1 0 0 1 0 0 0 0 1 1]  score:  27
[0 1 1 0 0 0 0 1 1 1]  score:  27
[1 1 0 1 0 0 0 1 0 1]  score:  31
[1 1 0 0 0 0 0 1 1 1]  score:  30
[0 1 0 1 0 0 0 0 1 1]  score:  32
[1 0 1 1 0 0 0 1 0 1]  score:  28
[1 1 0 1 0 0 0 0 1 1]  score:  25


AttributeError: 'list' object has no attribute 'shape'